<a href="https://colab.research.google.com/github/julinas/202-project3/blob/scene/cartoonGAN/RunCartoonGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title mount drive
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#@title cd to project3 folder
%cd 'drive/My Drive/Colab Notebooks/project3'

/content/drive/My Drive/Colab Notebooks/project3


In [0]:
#@title pytorch version run test
# import torch
# !python test.py --input_dir "../sample" --output_dir "../sample_out" --style Hosoda --gpu 0

In [3]:
#@title imports 
import cv2
import numpy as np
import os
import argparse
from PIL import Image
import tensorflow as tf

from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Flatten, Dense, BatchNormalization, ReLU, Add, LeakyReLU
from keras.models import Sequential
from keras.optimizers import *
from keras import Model

from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing import image
from keras import backend as K

Using TensorFlow backend.


In [0]:
#@title class Conv (Conv2D with padding="same")
class Conv(Conv2D):
  def __init__(self, filters, kernel_size, **kwargs):
    super(Conv, self).__init__(filters, kernel_size, padding="same", **kwargs)

In [0]:
#@title class ConvT (Conv2DTranspose with padding="same")
class ConvT(Conv2DTranspose):
  def __init__(self, filters, kernel_size, **kwargs):
    super(ConvT, self).__init__(filters, kernel_size, padding="same", **kwargs)

In [0]:
#@title class Norm (BatchNormalization with epsilon=1e-9)
class Norm(BatchNormalization): 
  def __init__(self, **kwargs):
    super(Norm, self).__init__(epsilon=1e-9, **kwargs)

In [0]:
#@title class LReLU (LeakyReLU with alpha=0.2)
class LReLU(LeakyReLU):
  def __init__(self, **kwargs):
    super(LReLU, self).__init__(alpha=0.2, **kwargs)


In [0]:
#@title def make_generator (use: generator = make_generator())

def make_generator():
  in_ = Input(shape=(256, 256, 3))
  conv1 = Conv(64, 7)(in_)
  norm1 = Norm()(conv1)
  relu1 = ReLU()(norm1)

  # Down-convolution
  conv2 = Conv(128, 3, strides=(2, 2))(relu1)
  conv3 = Conv(128, 3)(conv2)
  norm2 = Norm()(conv3)
  relu2 = ReLU()(norm2)

  conv4 = Conv(256, 3, strides=(2, 2))(relu2)
  conv5 = Conv(256, 3)(conv4)
  norm3 = Norm()(conv5)
  relu3 = ReLU()(norm3)

  # 8 residual blocks
  # block_1
  res_conv1_1 = Conv(256, 3)(relu3)
  res_norm_1 = Norm()(res_conv1_1)
  res_relu_1 = ReLU()(res_norm_1)
  res_conv2_1 = Conv(256, 3)(res_relu_1)
  res_norm2_1 = Norm()(res_conv2_1)
  res_add_1 = Add()([relu3, res_norm2_1])

  # block_2
  res_conv1_2 = Conv(256, 3)(res_add_1)
  res_norm_2 = Norm()(res_conv1_2)
  res_relu_2 = ReLU()(res_norm_2)
  res_conv2_2 = Conv(256, 3)(res_relu_2)
  res_norm2_2 = Norm()(res_conv2_2)
  res_add_2 = Add()([relu3, res_norm2_2])

  # block_3
  res_conv1_3 = Conv(256, 3)(res_add_2)
  res_norm_3 = Norm()(res_conv1_3)
  res_relu_3 = ReLU()(res_norm_3)
  res_conv2_3 = Conv(256, 3)(res_relu_3)
  res_norm2_3 = Norm()(res_conv2_3)
  res_add_3 = Add()([relu3, res_norm2_3])

  # block_4
  res_conv1_4 = Conv(256, 3)(res_add_3)
  res_norm_4 = Norm()(res_conv1_4)
  res_relu_4 = ReLU()(res_norm_4)
  res_conv2_4 = Conv(256, 3)(res_relu_4)
  res_norm2_4 = Norm()(res_conv2_4)
  res_add_4 = Add()([relu3, res_norm2_4])

  # block_5
  res_conv1_5 = Conv(256, 3)(res_add_4)
  res_norm_5 = Norm()(res_conv1_5)
  res_relu_5 = ReLU()(res_norm_5)
  res_conv2_5 = Conv(256, 3)(res_relu_5)
  res_norm2_5 = Norm()(res_conv2_5)
  res_add_5 = Add()([relu3, res_norm2_5])

  # block_6
  res_conv1_6 = Conv(256, 3)(res_add_5)
  res_norm_6 = Norm()(res_conv1_6)
  res_relu_6 = ReLU()(res_norm_6)
  res_conv2_6 = Conv(256, 3)(res_relu_6)
  res_norm2_6 = Norm()(res_conv2_6)
  res_add_6 = Add()([relu3, res_norm2_6])

  # block_7
  res_conv1_7 = Conv(256, 3)(res_add_6)
  res_norm_7 = Norm()(res_conv1_7)
  res_relu_7 = ReLU()(res_norm_7)
  res_conv2_7 = Conv(256, 3)(res_relu_7)
  res_norm2_7 = Norm()(res_conv2_7)
  res_add_7 = Add()([relu3, res_norm2_7])

  # block_8
  res_conv1_8 = Conv(256, 3)(res_add_7)
  res_norm_8 = Norm()(res_conv1_8)
  res_relu_8 = ReLU()(res_norm_8)
  res_conv2_8 = Conv(256, 3)(res_relu_8)
  res_norm2_8 = Norm()(res_conv2_8)
  res_add_8 = Add()([relu3, res_norm2_8])

  # Up-convolution
  upconv1 = ConvT(128, 3, strides=(2, 2))(res_add_8)
  upconv2 = Conv(128, 3)(upconv1)
  upnorm1 = Norm()(upconv2)
  uprelu1 = ReLU()(upnorm1)

  upconv3 = ConvT(64, 3, strides=(2, 2))(uprelu1)
  upconv4 = Conv(64, 3)(upconv3)
  upnorm2 = Norm()(upconv4)
  uprelu2 = ReLU()(upnorm2)

  out_ = Conv(3, 7)(uprelu2)
  return Model(in_, out_)

In [0]:
#@title def make_discriminator (use: discriminator = make_discriminator())

def make_discriminator():
  in_ = Input(shape=(256, 256, 3))
  conv1 = Conv(32, 3)(in_)
  relu1 = LReLU()(conv1)

  conv2 = Conv(64, 3, strides=(2, 2))(relu1)
  relu2 = LReLU()(conv2)
  conv3 = Conv(128, 3)(relu2)
  norm1 = Norm()(conv3)
  relu3 = LReLU()(norm1)

  conv4 = Conv(128, 3, strides=(2, 2))(relu3)
  relu4 = LReLU()(conv4)
  conv5 = Conv(256, 3)(relu4)
  norm2 = Norm()(conv5)
  relu5 = LReLU()(norm2)

  conv6 = Conv(256, 3)(relu5)
  norm3 = Norm()(conv6)
  relu6 = LReLU()(norm3)
  conv7 = Conv(1, 3)(relu6)
  
  flat = Flatten()(conv7)
  
  out_ = Dense(1, activation='softmax')(flat)
  return Model(in_, out_)


In [0]:
#@title def vggLoss
def vggLoss(a, b):
  vgg = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
#   a = a + tf.zeros((16, 256, 256, 3), dtype=a.dtype)
#   b = b + tf.zeros((16, 256, 256, 3), dtype=b.dtype)
#   diff = K.sum(a - b)
  diff = vgg.predict(a) - vgg.predict(b)
  return K.sum(K.abs(diff))


In [0]:
def initVggLoss(a, b):
  return K.sum(K.abs(a-b))

In [0]:
#@title def edgeLoss
def edgeLoss(a, b):
  pass

In [0]:
#@title def parseFramesFromVid #runs 
# def parseFramesFromVid(path):
#   vidcap = cv2.VideoCapture(path)
#   success,image = vidcap.read()
#   count = 0
#   while success:
#     cv2.imwrite("keras_files/dataset/train/frame%d.jpg" % count, image)
#     success,image = vidcap.read()
#     print(image.shape)
#     count += 1

# parseFramesFromVid('keras_files/tokyo.mp4') # this generates 6059 files

In [0]:
#@title def getDataFromPath
def getDataFromPath(path):
  data = []
  for (_, _, fnames) in os.walk(path):
    for fname in fnames:
      fpath = path + "/" + fname
      im = cv2.imread(fpath)
      data.append(im)
  return np.asarray(data)

In [0]:
#@title def getDataFromPathSingle (resizes and crops to center 256x256)
def getDataFromPathSingle(paths):
  data = []
  for path in paths:
    im = cv2.imread(path)
    width = int(256/im.shape[0]*im.shape[1]) # calculate goal width, assuming width>height
    im = cv2.resize(im, (width, 256)) # resize
    crop_index = int(im.shape[1]/2 - 256/2) # crop index so width is 256
    im = im[:, crop_index:crop_index+256] # crop
    data.append(im)
  return np.asarray(data)

In [0]:
# #@title class CartoonGAN
class CartoonGAN():
  def __init__(self):
    self.g = make_generator()
    self.d = make_discriminator()
    self.vgg = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    self.vgg.trainable = False
    for l in self.vgg.layers:
      l.trainable = False
    self.vggout = self.vgg(self.g.output)
    self.initModel = Model(self.g.input, self.vggout)
    self.initModel.compile(loss=initVggLoss, optimizer='adam')
    
  
  def pretrainGenerator(self, photos, epochs=5):
    # goal of this step is to reconstruct image with 
    # references: https://stackoverflow.com/questions/43914931/vgg-perceptual-loss-in-keras
    
    # adjust training data for combined model
    goal_pred = self.vgg.predict(photos)
    

    # train over combined model
    self.initModel.fit(photos, goal_pred, batch_size=16, epochs=epochs)
    self.g.save('keras_files/initializedGenerator.hdf5')
  
  def train(self, photos):
    pass
  
  def generate(self, photos):
    gen_pred = self.g.predict(photos)
    print(gen_pred.shape)
    for i in range(gen_pred.shape[0]):
      im_array = gen_pred[i]
      im = Image.fromarray((im_array * 255 / np.max(im_array)).astype('uint8'))
      im.save('keras_files/sample_out/' + str(i) + '.jpg')
    

In [0]:
photos = getDataFromPath('keras_files/dataset/train256x256')

In [0]:
#@title create GAN and run initializer
gan = CartoonGAN()
gan.pretrainGenerator(photos, epochs=20)

In [0]:
#@title test generate
testdata = getDataFromPathSingle(['keras_files/dataset/train/frame1545.jpg', 'keras_files/dataset/train/frame1546.jpg'])
gan.generate(testdata)

In [0]:
#@title train GAN
gan.train()